In [1]:
!pip install transformers

In [4]:
import torch
import json
import random
from torch.utils.data import TensorDataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW


tokenizer = ElectraTokenizer.from_pretrained("csebuetnlp/banglabert")


path= "/content/drive/MyDrive/Neural Network project/3lan.json"
with open(path, 'r', encoding='utf-8') as file:
    intents = json.load(file)["intents"]


patterns = []
responses = []

for intent in intents:
    patterns.extend(intent["patterns"])
    responses.extend(intent["responses"])

encoded_inputs = tokenizer(patterns, return_tensors="pt", padding=True, truncation=True, max_length=128)


labels = []
for intent_idx, intent in enumerate(intents):
    labels.extend([intent_idx] * len(intent["patterns"]))
labels = torch.tensor(labels)


dataset = TensorDataset(encoded_inputs["input_ids"], encoded_inputs["attention_mask"], labels)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

model = ElectraForSequenceClassification.from_pretrained("csebuetnlp/banglabert", num_labels=len(intents))



optimizer = AdamW(model.parameters(), lr=5e-5)

# Training
epochs = 10
for epoch in range(epochs):
    for batch in dataloader:
        inputs, attention_mask, label = batch
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Chat with the trained model
while True:
    user_input = input("You: ")
    if user_input == 'ধন্যবাদ':
        print("Bot: আপনাকেও ধন্যবাদ!")
        break


    encoded_user_input = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True,)


    model_output = model(**encoded_user_input)
    predicted_intent_idx = torch.argmax(model_output.logits).item()


    response_options = intents[predicted_intent_idx]["responses"]

    bot_response = response_options[random.randint(0,len(response_options))]
    print(f"Bot: {bot_response}")


You: হ্যালো


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Bot: হাই  %% USER %% ! আমি কীভাবে সাহায্য করতে পারি?
You: আমার মাথা বেথা, বমি বমি ভাব 
Bot: আপনার স্বাস্থ্য সমস্যা সম্পর্কে আমাকে বলুন
You: মাথা ঘুরে


IndexError: list index out of range